In [ ]:
!python -m pip install roboflow ultralytics
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-y_pmxyrs
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-y_pmxyrs
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
  Using cached yacs-0.1.8-py3-none-any.whl.metadata (639 bytes)
  Using cached fvcore-0.1.5.post20221221.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.7 MB/s eta 0:00:0000:01
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6361716 sha256=d9c6072be6f86351035920bd300fd849072ec3cf7317e962d0270a818cb4f069
  Stored in directory: /tmp/pip-ephem-wheel-cac

###Создание папки для сохранения результата

In [ ]:
import os

os.makedirs('/kaggle/working/output_mod_model_40000', exist_ok=True)

### Копирование ранее обученой модели в папку, меняя название на `model_pretrain.pth`

In [ ]:
import shutil

shutil.copy2('/kaggle/input/10k/pytorch/default/5/model_final.pth', '/kaggle/working/output_mod_model_40000/model_pretrain.pth')


'/kaggle/working/output_mod_model_40000/model_pretrain.pth'

### Поменять пути к датасету, а так же к чекпоинту(обученной ранее модели)

In [ ]:
import logging
import torch
from collections import OrderedDict
import os
import random
import numpy as np
from detectron2.utils.logger import setup_logger
setup_logger()

# Импорт библиотек Detectron2
import detectron2.utils.comm as comm
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
from detectron2.data import build_detection_train_loader, build_detection_test_loader
from detectron2.evaluation import (
    COCOEvaluator,
    inference_on_dataset,
    print_csv_format,
)

# Регистрация датасетов
register_coco_instances("animal-2_train", {}, f"/kaggle/input/animal-seg-1/animal-2-aug-1/train/_annotations.coco.json", f"/kaggle/input/animal-seg-1/animal-2-aug-1/train/")
register_coco_instances("animal-2_val", {}, f"/kaggle/input/animal-seg-1/animal-2-aug-1/valid/_annotations.coco.json", f"/kaggle/input/animal-seg-1/animal-2-aug-1/valid/")
register_coco_instances("animal-2_test", {}, f"/kaggle/input/animal-seg-1/animal-2-aug-1/test/_annotations.coco.json", f"/kaggle/input/animal-seg-1/animal-2-aug-1/test/")

# Настройка конфигурации
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
cfg.DATASETS.TRAIN = ("animal-2_train",)
cfg.DATASETS.TEST = ("animal-2_val",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = './output_mod_model_40000/model_pretrain.pth'
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.OPTIMIZER = "AdamW"
cfg.SOLVER.MAX_ITER = 40000

cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupMultiStepLR"
cfg.SOLVER.STEPS = [35000, 38000]
cfg.SOLVER.GAMMA = 0.01

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Количество классов в задаче
cfg.INPUT.MASK_FORMAT = "bitmask"
cfg.OUTPUT_DIR = "./output_mod_model_40000"
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Функция для оценки модели
def get_evaluator(cfg, dataset_name, output_folder=None):
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    return COCOEvaluator(dataset_name, cfg, True, output_folder)

# Функция для тестирования модели
def do_test(cfg, model):
    results = OrderedDict()
    for dataset_name in cfg.DATASETS.TEST:
        data_loader = build_detection_test_loader(cfg, dataset_name)
        evaluator = get_evaluator(cfg, dataset_name)
        results_i = inference_on_dataset(model, data_loader, evaluator)
        results[dataset_name] = results_i
        print_csv_format(results_i)
    return results

# Инициализация модели, оптимизатора и загрузка чекпоинта
resume = True
model = build_model(cfg)
optimizer = build_optimizer(cfg, model)
scheduler = build_lr_scheduler(cfg, optimizer)

# Загрузка чекпоинта(поменять путь)
checkpointer = DetectionCheckpointer(model, '/kaggle/working/output_mod_model_40000', optimizer=optimizer, scheduler=scheduler)

start_iter = 30000
if resume:
    start_iter = checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1
    if start_iter < 30000:  # Если загруженный чекпоинт меньше 15,000, принудительно установим 15,000
        start_iter = 30000
        print(1)

max_iter = cfg.SOLVER.MAX_ITER


patience_counter = 0
PATIENCE = 20000
BEST_LOSS = np.inf
prev_iter = start_iter
periodic_checkpointer = PeriodicCheckpointer(
    checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD, max_iter=max_iter
)
writers = default_writers(cfg.OUTPUT_DIR, max_iter) if comm.is_main_process() else []
data_loader = build_detection_train_loader(cfg)
logger = logging.getLogger("detectron2")
logger.info(f"Starting training from iteration {start_iter}")

with EventStorage(start_iter) as storage:
    for data, iteration in zip(data_loader, range(start_iter, max_iter)):
        storage.iter = iteration

        loss_dict = model(data)
        losses = sum(loss_dict.values())
        assert torch.isfinite(losses).all(), loss_dict

        loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())
        if comm.is_main_process():
            storage.put_scalars(total_loss=losses_reduced, **loss_dict_reduced)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
        scheduler.step()

        if (
            cfg.TEST.EVAL_PERIOD > 0
            and (iteration + 1) % cfg.TEST.EVAL_PERIOD == 0
            and iteration != max_iter - 1
        ):
            do_test(cfg, model)
            # Compared to "train_net.py", the test results are not dumped to EventStorage
            comm.synchronize()

        if iteration - start_iter > 5 and (
            (iteration + 1) % 20 == 0 or iteration == max_iter - 1
        ):
            for writer in writers:
                writer.write()
        periodic_checkpointer.step(iteration)

        if iteration > prev_iter:
            prev_iter = iteration
            if losses_reduced < BEST_LOSS:
                BEST_LOSS = losses_reduced
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter % 100 == 0:
                    print(f"Loss has not improved for {patience_counter} iterations")
                if patience_counter >= PATIENCE:
                    print(f"EARLY STOPPING")
                    break

# Финальное тестирование
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
do_test(cfg, model)

# Визуализация предсказаний
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("animal-2_val")
for d in random.sample(dataset_dicts, 3):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("animal-2_val"), scale=0.8, instance_mode=ColorMode.IMAGE_BW)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()


[02/09 05:03:47 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_mod_model_40000/model_pretrain.pth ...


/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[02/09 05:03:53 d2.data.datasets.coco]: Loading /kaggle/input/animal-seg-1/animal-2-aug-1/train/_annotations.coco.json takes 4.63 seconds.
WARNING [02/09 05:03:53 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

WARNING [02/09 05:03:53 d2.data.datasets.coco]: /kaggle/input/animal-seg-1/animal-2-aug-1/train/_annotations.coco.json contains 166074 annotations, but only 163276 of them match to images in the file.
[02/09 05:03:53 d2.data.datasets.coco]: Loaded 120927 images in COCO format from /kaggle/input/animal-seg-1/animal-2-aug-1/train/_annotations.coco.json
[02/09 05:03:55 d2.data.build]: Removed 69 images with no usable annotations. 120858 images left.
[02/09 05:03:58 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   | category   | #instances   |
|:----------:|:-------------|:-----------|:-------------|
|   animal   | 0            | 0          | 163276       |
|            |  

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/detectron2/data/detection_utils.py:449: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  torch.stack([torch.from_numpy(np.ascontiguousarr

[02/09 05:05:00 d2.utils.events]:  iter: 30019  total_loss: 0.332  loss_cls_stage0: 0.01676  loss_box_reg_stage0: 0.0145  loss_cls_stage1: 0.01445  loss_box_reg_stage1: 0.03723  loss_cls_stage2: 0.01278  loss_box_reg_stage2: 0.03976  loss_mask: 0.1405  loss_rpn_cls: 0.007707  loss_rpn_loc: 0.002513     lr: 1.9981e-05  max_mem: 11370M
[02/09 05:06:11 d2.utils.events]:  eta: 9:43:51  iter: 30039  total_loss: 0.3537  loss_cls_stage0: 0.02323  loss_box_reg_stage0: 0.01482  loss_cls_stage1: 0.02255  loss_box_reg_stage1: 0.03239  loss_cls_stage2: 0.01634  loss_box_reg_stage2: 0.04431  loss_mask: 0.1645  loss_rpn_cls: 0.01308  loss_rpn_loc: 0.003673     lr: 3.9961e-05  max_mem: 11691M
[02/09 05:07:07 d2.utils.events]:  eta: 7:50:53  iter: 30059  total_loss: 0.2467  loss_cls_stage0: 0.02025  loss_box_reg_stage0: 0.009962  loss_cls_stage1: 0.01758  loss_box_reg_stage1: 0.02221  loss_cls_stage2: 0.009784  loss_box_reg_stage2: 0.03685  loss_mask: 0.1041  loss_rpn_cls: 0.009084  loss_rpn_loc: 0.00

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[02/09 13:05:53 d2.evaluation.evaluator]: Start inference on 2560 batches
[02/09 13:05:58 d2.evaluation.evaluator]: Inference done 11/2560. Dataloading: 0.0011 s/iter. Inference: 0.3762 s/iter. Eval: 0.0025 s/iter. Total: 0.3797 s/iter. ETA=0:16:07
[02/09 13:06:03 d2.evaluation.evaluator]: Inference done 25/2560. Dataloading: 0.0016 s/iter. Inference: 0.3752 s/iter. Eval: 0.0022 s/iter. Total: 0.3792 s/iter. ETA=0:16:01
[02/09 13:06:08 d2.evaluation.evaluator]: Inference done 39/2560. Dataloading: 0.0018 s/iter. Inference: 0.3747 s/iter. Eval: 0.0024 s/iter. Total: 0.3790 s/iter. ETA=0:15:55
[02/09 13:06:13 d2.evaluation.evaluator]: Inference done 53/2560. Dataloading: 0.0018 s/iter. Inference: 0.3746 s/iter. Eval: 0.0022 s/iter. Total: 0.3787 s/iter. ETA=0:15:49
[02/09 13:06:19 d2.evaluation.evaluator]: Inference done 67/2560. Dataloading: 0.0018 s/iter. Inference: 0.3743 s/iter. Eval: 0.0021 s/iter. Total: 0.3784 s/iter. ETA=0:15:43
[02/09 13:06:24 d2.evaluation.evaluator]: Inference

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

WARNING [02/09 13:22:07 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/09 13:22:07 d2.data.datasets.coco]: Loaded 2560 images in COCO format from /kaggle/input/animal-seg-1/animal-2-aug-1/valid/_annotations.coco.json


NameError: name 'cv2' is not defined